### Rank Text Content by Semantic Similarity

In [35]:
import pandas as pd
import sys
import csv
import re
maxInt = sys.maxsize

while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
df_extracted = pd.read_csv('df_extracted.csv', engine='python', encoding='utf-8', error_bad_lines=False)

In [4]:
df_extracted.describe()

,Unnamed: 0,Unnamed: 0.1,count
count,18977.000000,18977.000000,1.897700e+04
mean,9488.000000,11927.675344,9.184705e+04
std,5478.332365,6883.190923,1.599096e+05
min,0.000000,0.000000,1.010000e+02
25%,4744.000000,5946.000000,1.025200e+04
50%,9488.000000,11961.000000,2.830500e+04
75%,14232.000000,17885.000000,1.055390e+05
max,18976.000000,23795.000000,2.708538e+06


In [69]:
df_extracted_sample = df_extracted[:5]
df_extracted_sample

,Unnamed: 0,Unnamed: 0.1,file_name,clean_text,count,similarity_index
0,0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf,corporate social responsibility performance re...,29122,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
1,1,1,20150416_COP_N°5_EN.pdf,sustainable development the energy that fuels ...,11939,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
2,2,2,2017_report_-_FINAL__UNLOCKED.pdf,the whole or any part of this work may not be ...,226975,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
3,3,3,COP_Report_SMART_-_Apr_2013_final.pdf,ptsinarmasagro resources andtechnology tbkglob...,37025,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
4,4,4,cop_ETS_2019.pdf,eser contracting and industry co. inc. communi...,35188,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."


In [6]:
!pip install scikit-learn
!pip install gensim
!pip install nltk

Document: a piece of text, in the form of a string. This could be just a few words, or a whole novel.  
Corpus: a collection of documents.  
Term: a word in a document.  

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

In [76]:
# Download stopwords list
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ohike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ohike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ohike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [77]:
# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

In [78]:
# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

In [79]:
search_term = ['conflict', 'violence', 'displacement', 'drought', 'earthquake', 'fire', 'flooding', 'freeze', 'health emergency', 'dengue', 'pneumonic plague', 'measles', 'landslide', 'tropical storm', 'typhoon', 'cyclone', 'hurricane', 'tsunami', 'urban disaster', 'volcanic eruption', 'refugee', 'terrorist attack', 'cold wave', 'complex emergency', 'epidemic', 'extratropical cyclone', 'flash flood', 'flood', 'heat wave', 'insect infestation', 'land slide', 'mud slide','snow avalanche', 'storm surge', 'technological disaster', 'tropical cyclone', 'volcano', 'wild fire', 'flood crisis', 'explosion', 'affected tornado', 'affected', 'death toll', 'tornado relief', 'flood appeal', 'massive explosion', 'affected areas', 'praying victims', 'injured', 'lurches fire', 'flood relief', 'flood affected', 'tornado victims', 'deadly', 'evacuated', 'relief', 'flood death', 'deaths confirmed', 'affected flooding', 'people killed', 'flood damage', 'people dead', 'major flood', 'rubble', 'another explosion', 'flood warnings', 'tornado survivor', 'damage', 'devastating', 'flood toll', 'affected hurricane', 'crisis', 'relief efforts', 'flood emergency', 'fire flood', 'huge explosion', 'bushfire', 'torrential rains', 'affected explosion', 'disaster', 'twister', 'blast', 'injuries reported', 'fatalities', 'large explosion', 'destroyed', 'displaced', 'casualties', 'climate change', 'major explosion', 'response disasters', 'explosion victims', 'tragic', 'dealing hurricane', 'flood recovery', 'dead torrential', 'flood years', 'massive tornado', 'buried alive', 'alive rubble', 'crisis rises', 'flood ravaged', 'killed injured', 'killed people', 'people died', 'floods kill', 'tornado damage', 'facing flood', 'deadly explosion', 'flood disaster', 'tornado disaster', 'help victims', 'hundreds homes', 'severe flooding', 'magnitude', 'firefighters police', 'fire explosion', 'storm', 'flood hits', 'floodwaters', 'emergency', 'flood alerts', 'crisis unfolds', 'tragic events', 'deadly tornado', 'people trapped', 'surging floods', 'city tornado', 'damaged hurricane', 'rains severely', 'house flood', 'devastating tornado', 'lost lives', 'reportedly dead', 'following explosion', 'tornado flood', 'early warninig', 'warning', 'dead floods', 'flood threat', 'flood situation', 'risk running', 'loss life', 'thoughts victims', 'terrible explosion', 'seismic', 'flood homeowners', 'flood claims', 'power supplies', 'free hotline', 'hotline help', 'registered magnitude', 'prepare hurricane', 'landfall', 'crisis worsens', 'communities damaged', 'destruction', 'tornado', 'hurricane coming', 'toxins flood', 'release toxins', 'toxins', 'supplies waters', 'crisis found', 'braces major', 'government negligent', 'attack', 'waiting hurricane', 'terror', 'memorial service', 'terror attack', 'coast hurricane', 'terrified hurricane', 'hurricane category', 'disaster relief', 'cleanup', 'troops lend', 'effected hurricane', 'time hurricane', 'saying hurricane', 'praying families', 'dramatic', 'path hurricane']
search_term_dic = {i: search_term[i] for i in range(len(search_term))}
search_term = search_term_dic

In [ ]:
vectorizer = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 4),
                             max_df = .75, min_df = 1, max_features = 100000)
vectorizer.fit(search_term.values())

M = vectorizer.fit_transform(df_extracted['clean_text'])

df_keywords = pd.DataFrame(vectorizer.transform(search_term.values()).todense(),
                          index = search_term.keys(),
                          columns = vectorizer.get_feature_names())
df_keywords.index.name = 'report'

print('Shape:', df_keywords.shape)
display(df_keywords.head())

In [81]:
sorted(vectorizer.vocabulary_.items())

[('affected', 0),
 ('affected areas', 1),
 ('affected explosion', 2),
 ('affected flooding', 3),
 ('affected hurricane', 4),
 ('affected tornado', 5),
 ('alerts', 6),
 ('alive', 7),
 ('alive rubble', 8),
 ('appeal', 9),
 ('areas', 10),
 ('attack', 11),
 ('avalanche', 12),
 ('blast', 13),
 ('braces', 14),
 ('braces major', 15),
 ('buried', 16),
 ('buried alive', 17),
 ('bushfire', 18),
 ('casualties', 19),
 ('category', 20),
 ('change', 21),
 ('city', 22),
 ('city tornado', 23),
 ('claims', 24),
 ('cleanup', 25),
 ('climate', 26),
 ('climate change', 27),
 ('coast', 28),
 ('coast hurricane', 29),
 ('cold', 30),
 ('cold wave', 31),
 ('coming', 32),
 ('communities', 33),
 ('communities damaged', 34),
 ('complex', 35),
 ('complex emergency', 36),
 ('confirmed', 37),
 ('conflict', 38),
 ('crisis', 39),
 ('crisis rises', 40),
 ('crisis unfolds', 41),
 ('crisis worsens', 42),
 ('cyclone', 43),
 ('damage', 44),
 ('damaged', 45),
 ('damaged hurricane', 46),
 ('dead', 47),
 ('dead floods', 48),


In [86]:
#for i in range(len(df_extracted['clean_text'])):
#    file_name = df_extracted['file_name'][i]
#    clean_text = df_extracted['clean_text'][i]
#    
#    doc_vectors = vectorizer.fit_transform('clean_text')
#    cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
#
#    df_extracted['similarity_index'][i] = cosine_similarities

ValueError: Iterable over raw text documents expected, string object received.

In [87]:
#doc_vectors = vectorizer.fit_transform(df_extracted['clean_text'])
#cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()

#df_extracted['similarity_index'] = cosine_similarities

KeyboardInterrupt: 

In [ ]:
df_similarity = pd.DataFrame(cosine_similarity(X = df_keywords.values, Y = M).T,
                             columns = df_keywords.index.tolist())
print('Shape:', df_similarity.shape)
display(df_similarity.head())

In [89]:
df_extracted

,Unnamed: 0,Unnamed: 0.1,file_name,clean_text,count,similarity_index
0,0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf,corporate social responsibility performance re...,29122,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
1,1,1,20150416_COP_N°5_EN.pdf,sustainable development the energy that fuels ...,11939,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
2,2,2,2017_report_-_FINAL__UNLOCKED.pdf,the whole or any part of this work may not be ...,226975,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
3,3,3,COP_Report_SMART_-_Apr_2013_final.pdf,ptsinarmasagro resources andtechnology tbkglob...,37025,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
4,4,4,cop_ETS_2019.pdf,eser contracting and industry co. inc. communi...,35188,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
...,...,...,...,...,...,...
18972,18972,23789,CSR_Report_2014.pdf,ah industries corporate social r esponsibility...,39306,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
18973,18973,23792,Nilorngruppen_Sustainability_report.pdf,1more fit for the future sustainability report...,112118,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
18974,18974,23793,MM_CSR_2018.pdf,annual corporate social responsibility report ...,39360,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."
18975,18975,23794,AmatheonAgri-COP-2017.pdf,01amatheon agri communication on progress 2017...,21804,"(0, 615)\t0.14519212172971319\n (0, 612)\t0..."


In [8]:
df_extracted.to_csv('df_extracted_ranked.csv')

In [9]:
pd.read_csv('df_extracted_ranked.csv', engine='python', encoding='utf-8', error_bad_lines=False)

,Unnamed: 0,Unnamed: 0.1,file_name,clean_text,count
0,0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf,corporate social responsibility performance re...,29122
1,1,1,20150416_COP_N°5_EN.pdf,sustainable development the energy that fuels ...,11939
2,2,2,2017_report_-_FINAL__UNLOCKED.pdf,the whole or any part of this work may not be ...,226975
3,3,3,COP_Report_SMART_-_Apr_2013_final.pdf,ptsinarmasagro resources andtechnology tbkglob...,37025
4,4,4,cop_ETS_2019.pdf,eser contracting and industry co. inc. communi...,35188
...,...,...,...,...,...
18972,18972,23789,CSR_Report_2014.pdf,ah industries corporate social r esponsibility...,39306
18973,18973,23792,Nilorngruppen_Sustainability_report.pdf,1more fit for the future sustainability report...,112118
18974,18974,23793,MM_CSR_2018.pdf,annual corporate social responsibility report ...,39360
18975,18975,23794,AmatheonAgri-COP-2017.pdf,01amatheon agri communication on progress 2017...,21804
